In [1]:
import pandas as pd
import numpy as np
import math
import atoti as tt

Welcome to atoti 0.7.1!

By using this community edition, you agree with the license available at https://docs.atoti.io/latest/eula.html.
Browse the official documentation at https://docs.atoti.io.
Join the community at https://www.atoti.io/register.

atoti collects telemetry data, which is used to help understand how to improve the product.
If you don't wish to send usage data, set the ATOTI_DISABLE_TELEMETRY environment variable to True.

You can hide this message by setting the ATOTI_HIDE_EULA_MESSAGE environment variable to True.


## Load Data

In [2]:
courses_frame = pd.read_csv('data/courses.csv')

In [3]:
std_info_frame = pd.read_csv('data/studentInfo.csv')

In [4]:
assesments_frame = pd.read_csv('data/assessments.csv')

In [5]:
stdAsses_frame = pd.read_csv('data/studentAssessment.csv')

## Ambil fitur-fitur pada tabel yang akan untuk dianalisis

Tabel yang dipakai ada 4, yaitu course, studentInfo, assessment, dan studentAssesment

#### 1. Ambil fitur dari Tabel course

In [6]:
# ambil feature yang diperlukan pada dataframe std_asses_frame
courses = pd.DataFrame(courses_frame, columns=['code_presentation', 'code_module', 'module_presentation_length'])

# ubah tipe data dari int64 menjadi int32
courses['module_presentation_length'] = courses['module_presentation_length'].astype('int')

# tampilkan data
courses.head(5)

,code_presentation,code_module,module_presentation_length
0,2013J,AAA,268
1,2014J,AAA,269
2,2013J,BBB,268
3,2014J,BBB,262
4,2013B,BBB,240


#### 2. Ambil fitur dari Tabel studentInfo

In [7]:
# ambil feature yang diperlukan pada dataframe std_asses_frame
stdInfo = pd.DataFrame(std_info_frame, columns=['id_student', 'code_module', 'final_result', 'gender','imd_band',
                                                'highest_education', 'age_band', 'num_of_prev_attempts',
                                                'studied_credits', 'region'])

# id perlu diubah menjadi string
stdInfo['id_student'] = stdInfo['id_student'].astype('string')

# tampilkan data
stdInfo.head(5)


,id_student,code_module,final_result,gender,imd_band,highest_education,age_band,num_of_prev_attempts,studied_credits,region
0,11391,AAA,Pass,M,90-100%,HE Qualification,55<=,0,240,East Anglian Region
1,28400,AAA,Pass,F,20-30%,HE Qualification,35-55,0,60,Scotland
2,30268,AAA,Withdrawn,F,30-40%,A Level or Equivalent,35-55,0,60,North Western Region
3,31604,AAA,Pass,F,50-60%,A Level or Equivalent,35-55,0,60,South East Region
4,32885,AAA,Pass,F,50-60%,Lower Than A Level,0-35,0,60,West Midlands Region


#### 3. Ambil fitur dari Tabel assesments

In [8]:
# ambil feature yang diperlukan pada dataframe assesments_frame
asses = pd.DataFrame(assesments_frame, columns=['id_assessment', "code_module", "code_presentation", 'assessment_type', 'date'])

# id perlu diubah menjadi string
asses['id_assessment'] = asses['id_assessment'].astype('string')

asses.head(5)

,id_assessment,code_module,code_presentation,assessment_type,date
0,1752,AAA,2013J,TMA,19.0
1,1753,AAA,2013J,TMA,54.0
2,1754,AAA,2013J,TMA,117.0
3,1755,AAA,2013J,TMA,166.0
4,1756,AAA,2013J,TMA,215.0


#### 4. Ambil fitur dari Tabel studentAssesment

In [9]:
# ambil feature yang diperlukan pada dataframe assesments_frame
stdAsses = pd.DataFrame(stdAsses_frame, columns=['id_assessment', 'id_student', 'score', 'date_submitted', 'is_banked'])

# id perlu diubah menjadi string
stdAsses['id_assessment'] = stdAsses['id_assessment'].astype('string')
stdAsses['id_student'] = stdAsses['id_student'].astype('string')

stdAsses.head(5)

,id_assessment,id_student,score,date_submitted,is_banked
0,1752,11391,78.0,18,0
1,1752,28400,70.0,22,0
2,1752,31604,72.0,17,0
3,1752,32885,69.0,26,0
4,1752,38053,79.0,19,0


## Handling Null Values

#### Null values pada tabel course

In [10]:
pd.DataFrame(courses.isnull().sum().sort_values(ascending=False)).head()

,0
code_presentation,0
code_module,0
module_presentation_length,0


tidak ditemukan null values --> aman

#### Null values pada tabel studentInfo

In [11]:
pd.DataFrame(stdInfo.isnull().sum().sort_values(ascending=False)).head()

,0
imd_band,1111
id_student,0
code_module,0
final_result,0
gender,0


kolom imd_band akan di-drop saja karena tidak digunakan dalam analisis.

In [12]:
stdInfo = stdInfo.drop(['imd_band'], axis=1)

In [13]:
pd.DataFrame(stdInfo.isnull().sum().sort_values(ascending=False)).head()

,0
id_student,0
code_module,0
final_result,0
gender,0
highest_education,0


imd_band sudah di-drop

#### Null values pada tabel assesments

In [14]:
pd.DataFrame(asses.isnull().sum().sort_values(ascending=False)).head()

,0
date,11
id_assessment,0
code_module,0
code_presentation,0
assessment_type,0


assesment_type "Exam" ternyata date-nya null semua. Akan tetapi hal ini tidak masalah untuk dibiarkan karena date tidak dipakai dalam analisis. 

#### Null values pada tabel studentAssesment

In [15]:
pd.DataFrame(stdAsses.isnull().sum().sort_values(ascending=False)).head()

,0
score,173
id_assessment,0
id_student,0
date_submitted,0
is_banked,0


Fill missing values dengan mean dari score

In [16]:
stdAsses["score"]=stdAsses["score"].fillna(np.mean(stdAsses["score"]))

Cek ulang missing values

In [17]:
pd.DataFrame(stdAsses.isnull().sum().sort_values(ascending=False)).head()

,0
id_assessment,0
id_student,0
score,0
date_submitted,0
is_banked,0


Sudah tidak ada missing values

## Create Session

In [18]:
session = tt.Session()

# Cube ke-1

Memuat table Courses dan Student Info

### Load data dari pandas frame ke sebuah in-memory yang disebut "table"

#### course_table

In [19]:
course_table = session.read_pandas(courses, table_name="Courses")
course_table.head()

,code_presentation,code_module,module_presentation_length
0,2013J,AAA,268
1,2014B,EEE,241
2,2014J,AAA,269
3,2014J,CCC,269
4,2013J,EEE,268


#### stdInfo_table

In [20]:
stdInfo_table = session.read_pandas(stdInfo, table_name="Student Info")
stdInfo_table.head()

,id_student,code_module,final_result,gender,highest_education,age_band,num_of_prev_attempts,studied_credits,region
0,11391,AAA,Pass,M,HE Qualification,55<=,0,240,East Anglian Region
1,38053,AAA,Pass,M,A Level or Equivalent,35-55,0,60,Wales
2,59185,AAA,Pass,M,Lower Than A Level,35-55,0,60,East Anglian Region
3,102806,AAA,Pass,M,A Level or Equivalent,0-35,0,60,North Western Region
4,111717,AAA,Pass,F,HE Qualification,35-55,0,60,East Anglian Region


#### asses_table

In [21]:
asses_table = session.read_pandas(asses, table_name="Assessments")
asses_table.head()

,id_assessment,code_module,code_presentation,assessment_type,date
0,1752,AAA,2013J,TMA,19.0
1,14992,BBB,2013B,CMA,89.0
2,14986,BBB,2013B,TMA,89.0
3,14987,BBB,2013B,TMA,124.0
4,14989,BBB,2013B,TMA,187.0


#### stdAsses_table

In [22]:
stdAsses_table = session.read_pandas(stdAsses, table_name="Student Assessment")
stdAsses_table.head()

,id_assessment,id_student,score,date_submitted,is_banked
0,1752,11391,78.0,18,0
1,1752,100893,63.0,17,0
2,1752,106247,67.0,32,0
3,1752,127582,69.0,26,0
4,1752,142326,65.0,17,0


# Join Tables

Lakukan join table Courses dan Student Info untuk membuat cube

In [23]:
stdInfo_table.join(course_table, mapping={"code_module": "code_module"})

In [24]:
stdInfo_table.head()

,id_student,code_module,final_result,gender,highest_education,age_band,num_of_prev_attempts,studied_credits,region
0,11391,AAA,Pass,M,HE Qualification,55<=,0,240,East Anglian Region
1,38053,AAA,Pass,M,A Level or Equivalent,35-55,0,60,Wales
2,59185,AAA,Pass,M,Lower Than A Level,35-55,0,60,East Anglian Region
3,102806,AAA,Pass,M,A Level or Equivalent,0-35,0,60,North Western Region
4,111717,AAA,Pass,F,HE Qualification,35-55,0,60,East Anglian Region


### Create Cube

In [25]:
cube1 = session.create_cube(stdInfo_table)

In [26]:
# Aliasing the hierarchies property to a shorter variable name because we will use it a lot.
h = cube1.hierarchies
l = cube1.levels
m = cube1.measures

Berikut adalah Dimensions dan Measures dari cube1

In [27]:
cube1

Cube(_name='Student Info', _hierarchies=Hierarchies(_create_hierarchy_from_arguments=<bound method Cube._create_hierarchy_from_arguments of ...>, _cube_name='Student Info'), _measures=Measures())

## Measures, Slice, dan Dice

Coba mencari measure berupa: Mean dari studied_credits (SKS yang diambil):

In [28]:
cube1.query(m["studied_credits.MEAN"])

,studied_credits.MEAN
0,79.76


#### DICE untuk Mean studied_credits per Jenis final_result

Coba DICE cube untuk mencari measure berupa: Mean dari studied_credits (SKS yang diambil) tiap jenis final_result:

In [29]:
cube1.query(m["studied_credits.MEAN"], levels=[l["final_result"]])

,studied_credits.MEAN
final_result,
Distinction,71.76
Fail,76.09
Pass,74.22
Withdrawn,91.43


#### Ditemukan insight:

Pada tabel di atas dapat dilihat bahwa student yang mengambil lebih banyak SKS, akan rentan Withdrawn (mengundurkan diri) dan Fail (gagal).

Sementara itu, terlihat bahwa student dengan predikat Distinction (Istimewa) dan Pass (Lulus), mengambil lebih sedikit SKS.

#### SLICE dan DICE untuk melihat banyak student withdrawn pada tiap code_module dan code_presentation

Pada  tabel di bawah menunjukkan SKS/credits yang diambil oleh student yang Withdrawn pada beberapa module.

In [30]:
cube1.query(
    m["studied_credits.MEAN"],
    levels=[l["code_module"], l["code_presentation"]],
    filter=l["final_result"] == "Withdrawn",
)

,,studied_credits.MEAN
code_module,code_presentation,
AAA,2013J,102.86
BBB,2014B,91.82
CCC,2014J,85.51
DDD,2014J,98.57
EEE,2014B,74.60
FFF,2014B,100.11
GGG,2014J,38.51


#### Agar slice dan dice menjadi lebih fleksibel, maka daripada menggunakan code statis, gunakan fitur atoti untuk tabel dinamis

In [31]:
session.visualize()

Open the notebook in JupyterLab with the atoti extension enabled to build this widget.

##### Didapat insight:

Pada pivot table di atas, terlihat bahwa, hanya pada student dengan umur => 55 tahun, student yang Fail rata-rata hanya mengambil sedikit SKS. Terlihat bahwa pada usia =>55 tahun, pemegaruhnya tidak hanya jumlah SKS yang diambil. Bisa jadi karena faktor lain.

### Drilldown pada Cube 1

- Slice - ambil code_presentation tahun 2014 saja: 2014B dan 2014J
- Drill down dari code_presentation ke code_module
- Drill down dari code_module ke final_result

In [32]:
session.visualize()

Open the notebook in JupyterLab with the atoti extension enabled to build this widget.

In [33]:
session.visualize()

Open the notebook in JupyterLab with the atoti extension enabled to build this widget.

# Cube ke-2

Memuat 3 tables: assessments, course, dan studentAssessment

#### Load data dari pandas frame ke sebuah in-memory yang disebut "table"

#### stdInfo_table_2

In [34]:
stdInfo_table_2 = session.read_pandas(stdInfo, table_name="Student Info 2")
stdInfo_table_2.head(2)

,id_student,code_module,final_result,gender,highest_education,age_band,num_of_prev_attempts,studied_credits,region
0,11391,AAA,Pass,M,HE Qualification,55<=,0,240,East Anglian Region
1,38053,AAA,Pass,M,A Level or Equivalent,35-55,0,60,Wales


#### asses_table_2

In [35]:
asses_table_2 = session.read_pandas(asses, table_name="Assessments 2")
asses_table_2.head(2)

,id_assessment,code_module,code_presentation,assessment_type,date
0,1752,AAA,2013J,TMA,19.0
1,14992,BBB,2013B,CMA,89.0


#### stdAsses_table_2

In [36]:
stdAsses_table_2 = session.read_pandas(stdAsses, table_name="Student Assessment 2")
stdAsses_table_2.head(2)

,id_assessment,id_student,score,date_submitted,is_banked
0,1752,11391,78.0,18,0
1,1752,100893,63.0,17,0


#### course_table_2

In [37]:
course_table_2 = session.read_pandas(courses, table_name="Courses 2")
course_table_2.head(2)

,code_presentation,code_module,module_presentation_length
0,2013J,AAA,268
1,2014B,EEE,241


### Join Tables

Join 3 tables: assessments, course, dan studentAssessment

In [38]:
asses_table_2.join(course_table_2, mapping={"code_module": "code_module"})

In [39]:
stdAsses_table_2.join(asses_table_2, mapping={"id_assessment": "id_assessment"})

### Create Cube

In [40]:
cube2 = session.create_cube(stdAsses_table_2)

In [41]:
# Aliasing the hierarchies property to a shorter variable name because we will use it a lot.
h2, l2, m2 = cube2.hierarchies, cube2.levels, cube2.measures

Berikut adalah dimensions dan measures dari cube2

In [42]:
cube2

Cube(_name='Student Assessment 2', _hierarchies=Hierarchies(_create_hierarchy_from_arguments=<bound method Cube._create_hierarchy_from_arguments of ...>, _cube_name='Student Assessment 2'), _measures=Measures())

### Operasikan dan Visualisasikan

#### Roll Up pada cube 2

- Berikut merupakan tabel score.MEAN() tiap assessment_type.
- Tiap code_module punya 3 jenis assesment_type.
- Maka dapat di-roll up menjadi score.MEAN() terhadap code_module.

In [43]:
session.visualize()

Open the notebook in JupyterLab with the atoti extension enabled to build this widget.

#### Pivot Table mean score dari tiap code_module dan tiap code_presentation

In [44]:
session.visualize()

Open the notebook in JupyterLab with the atoti extension enabled to build this widget.

In [45]:
session.visualize()

Open the notebook in JupyterLab with the atoti extension enabled to build this widget.

Pada tabel di atas dapat dilihat bahwa assesment score pda module AAA dan DDD masih rendah.
Kedua module tersebut dapat diperhatikan lagi mengapa hasilnya kurang maksimal.

Apakah ada kekurangan dalam pengajaran, pemenuhan materi, atau pelaksanaan ujian.

Coba cari nilai tiap jenis ujian pada code_module AAA dan DDD, seperti di bawah.

#### Pivot Table dan Chart mean score dari tiap assesment_type pada code_module AAA dan DDD

In [46]:
session.visualize()

Open the notebook in JupyterLab with the atoti extension enabled to build this widget.

In [47]:
session.visualize()

Open the notebook in JupyterLab with the atoti extension enabled to build this widget.

Pada pivot table dan chart di atas terlihat assesment_type "Exam" memiliki nilai yang relatif rendah. Maka bisa dipelajari lebih lanjut mengapa hasil Exam student masih rendah.

#### Pivot Table mean date_submitted dari tiap jenis assessment_type

In [48]:
session.visualize()

Open the notebook in JupyterLab with the atoti extension enabled to build this widget.

Pada table di atas, terlihat bahwa Exam adalah jenis assessment yang paling lama dikumpulkan.
TMA (Tutor Marked Assessment) adalah yang paling cepat dikumpulkan.

Di pivot table selanjutnya dilihat pula berdasarkan code_module

#### Pivot Table mean date_submitted dari tiap module, dan dilihat juga berdasarkan assessment_type (di-slice untuk 2014 saja)

In [49]:
session.visualize()

Open the notebook in JupyterLab with the atoti extension enabled to build this widget.

In [50]:
session.visualize()

Open the notebook in JupyterLab with the atoti extension enabled to build this widget.

Pada pivot table dan chart di atas terlihat rincian lama waktu submisi dari tiap jenis ujian, untuk masing-masing code_module